In [1]:
import os

In [2]:
%pwd

'd:\\mjourFinalPorject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mjourFinalPorject'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/rashmi0710/Classfication-deep-learning-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="rashmi0710"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e711db8832861f8ebed2f9e68e91430cea19b918"

ModuleNotFoundError: No module named 'dagshub'